In [25]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
   
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if "Period_gene_genomic_sequence_individual_exon" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp")

    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa"')
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/temp/temp_genome.fa", "fasta")
    # genome = SeqIO.parse("/mnt/c/Users/sauba/Downloads/Compressed/ncbi_dataset_7/ncbi_dataset/data/GCA_917051295.2/GCA_917051295.2_ilEreLige1.2_genomic.fna", 'fasta')
    for entries in genome:
        gene_sequence = entries.seq
#         if complement == "1":
#             gene_sequence = gene_sequence.reverse_complement()
        break
    return (gene_sequence)

def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)   

def get_sequence(blast_output_location,genome_location, species, genome_file,annotated_genome_location,annotated_species_name):
    list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
    coordinate_file_name = ''
    for file_names in list_of_files_in_species_folder:
        if file_names.endswith("_coordinates.csv"):
            coordinate_file_name = file_names

    with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
        coordinate_file_lines = temp_file_open.readlines()

    upstream_exon_line =coordinate_file_lines[1].split(",")
    if upstream_exon_line[5] == "Y":
        print("First Exon Error!!")
        assert False

    downstream_exon_line = coordinate_file_lines[-1].split(",")
    if downstream_exon_line[5] == "Y":
        print("Last Exon Error!!")
        assert False

    
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3])) + 100000
        
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))

    if gene_start < 100000:
            print(f"Error{gene_start}")
    gene_start -= 100000
    gene_end +=  100000
    
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    print("BB")
    return(gene_sequence,scaffold,gene_start,gene_end )

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False



In [26]:
def make_species_folder(output_location, species):
    list_of_folders = os.listdir(f"{output_location}/5.For Email_padded/")
    if species not in list_of_folders:
        os.mkdir(f"{output_location}/5.For Email_padded/{species}")


In [27]:
import os
import io
import subprocess
genome_location =  "/mnt/f/Genomes_2023-12-26"
# genome_location =  "/mnt/g/Genomes_2023-12-26"
blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
# blast_output_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
list_of_species = os.listdir(f"{output_location}/2.Final_output")
list_of_species.remove('desktop.ini')
# print(list_of_species)
output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine"

for species in list_of_species:
    make_species_folder(output_location, species)
    annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
    # annotated_genome_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"

    genome_file = get_genome_file(genome_location,species)
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)

    gene_sequence,scaffold, gene_start, gene_end = get_sequence(blast_output_location,genome_location, species, genome_file,
                                                               annotated_genome_location,annotated_species_name)

#     print(gene_sequence,scaffold, gene_start, gene_end)
    with open(f"{output_location}/5.For Email_padded/{species}/Genome_fragment.fa",'w') as out_file:
        output = f">{species}_{scaffold}_{gene_start}_{gene_end}\n{gene_sequence}"
        out_file.write(output)

#     local_genomic_fragment_location = f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon/"
#     cd_command = f'cd "{local_genomic_fragment_location}"\nmakeblastdb -in gene_sequence_all.fa -dbtype nucl\n'
#         # # os.system(f'{cd_command}')
#     subprocess.run(f'{cd_command}', shell = True, stderr = subprocess.DEVNULL)
#         # # print(mkdb_command)
#         # # os.system(f'{mkdb_command}')
#     # print(gene_sequence)

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


...

BB


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


BB


In [35]:
import os
from Bio import SeqIO

output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine"

list_of_species = os.listdir(f"{output_location}/2.Final_output")
list_of_species.remove('desktop.ini')
# print(list_of_species)

for species in list_of_species:
    print(species)
#     print(gene_start)
    make_species_folder(output_location, species)
    output = ''
    genome_fragment_file = SeqIO.parse(f"{output_location}/5.For Email_padded/{species}/Genome_fragment.fa", "fasta")
    for sequences in genome_fragment_file:
        genome_name = sequences.id
    list_of_variant_files = os.listdir(f"{output_location}/4.Bed File/{species}/")
    if 'desktop.ini' in list_of_variant_files:
        list_of_variant_files.remove('desktop.ini')
#     start = 0
    
    for variant in list_of_variant_files:    
        with open(f"{output_location}/4.Bed File/{species}/{variant}", 'r') as var_file:
            var_list = var_file.readlines()
        
        if int((var_list[0].split("\t")[1])) <int((var_list[1].split("\t")[1])):
            complement = 0
            gene_start = int((var_list[0].split("\t")[1]))
        else:
            complement = 1
            gene_start = int((var_list[-1].split("\t")[1])) 
        output = ''
#         print(var_list)
        for gene_line in  var_list:
            gene_line_split = gene_line.split("\t")
#             print(gene_line, var_list)
            gene_line_split[0] = genome_name
            gene_line_split[1] = str(int(gene_line_split[1]) - gene_start + 100000)
            gene_line_split[2] = str(int(gene_line_split[2]) - gene_start + 100000)
            output += "\t".join(gene_line_split)
        
#         print(output)
        with open(f"{output_location}/5.For Email_padded/{species}/{variant}", 'w') as out_file:
            out_file.write(output)
#         print(complement, gene_start)
#         assert False
        

Bicyclus_anynana
Coenonympha_glycerion
Elymnias_hypermnestra
Erebia_aethiops
Erebia_ligea
Hipparchia_semele
Lasiommata_megera
Maniola_hyperantus
Maniola_jurtina
Melanargia_galathea
Oeneis_ivallda
Pararge_aegeria
